In [1]:
import pandas as pd


    header = None — загрузить без строки с заголовком;
    skiprows=n — пропустить n строк (часто у документов бывает техническая «шапка»);
    encoding — загрузить в конкретной кодировке;
    na_values — список значений, который нужно заменить на NaN (специальный объект, обозначающий пропущенное значение).


In [34]:
log = pd.read_csv("data/log.csv", header = None)

In [35]:
users = pd.read_csv("data/users.csv", encoding="KOI8-R", sep="	")

In [36]:
log.columns = ['user_id','time','bet','win']

In [37]:
users.columns = ['user_id','email','geo']

In [8]:
log.head()

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN


In [38]:
log_win = log[log['win'] > 0]

In [39]:
win_count = log_win.shape[0]
print(win_count)

138


In [40]:
log2 = log.query("bet<2000 & win>0")

In [41]:
log2

,user_id,time,bet,win
151,Запись пользователя № - user_982,[2019-01-16 21:54:22,100.0,4749.0
189,Запись пользователя № - user_964,[2019-01-21 18:34:44,200.0,4667.0
205,Запись пользователя № - user_931,[2019-01-22 5:26:59,300.0,4319.0
232,Запись пользователя № - user_998,[2019-01-25 8:57:20,500.0,5069.0
265,Запись пользователя № - user_998,[2019-01-29 10:37:55,500.0,6294.0
...,...,...,...,...
962,Запись пользователя № - user_976,[2019-04-19 14:18:45,400.0,5997.0
965,Запись пользователя № - user_900,[2019-04-19 19:31:48,900.0,6767.0
967,Запись пользователя № - user_975,[2019-04-19 22:25:15,1000.0,6108.0
981,#error,NaN,800.0,7035.0


In [42]:
log.user_id.unique()

array(['Запись пользователя № - user_919',
       'Запись пользователя № - user_973',
       'Запись пользователя № - user_903',
       'Запись пользователя № - user_954',
       'Запись пользователя № - user_917',
       'Запись пользователя № - user_900',
       'Запись пользователя № - user_980',
       'Запись пользователя № - user_906',
       'Запись пользователя № - user_952',
       'Запись пользователя № - user_972',
       'Запись пользователя № - user_966',
       'Запись пользователя № - user_964',
       'Запись пользователя № - user_978',
       'Запись пользователя № - user_968',
       'Запись пользователя № - user_974',
       'Запись пользователя № - user_998',
       'Запись пользователя № - user_990',
       'Запись пользователя № - user_942',
       'Запись пользователя № - user_929',
       'Запись пользователя № - user_976',
       'Запись пользователя № - user_927',
       'Запись пользователя № - user_951',
       'Запись пользователя № - user_949',
       'Зап

In [43]:
new_log = log[~log.user_id.str.contains("error", na=False)]

In [44]:
def get_user_id(s):
    pos = s.find("user")
    if pos > -1:
        return s[pos:]
    else:
        return ""

In [45]:
log['user_id'] = log.user_id.apply(get_user_id)


In [56]:
#t = log.time[0]
log['time'] = log[log.time.str.match("\[", na=False)]['time'].apply(lambda t: str(t).replace('[', ''))
#t = t[1:]

In [57]:
log

,user_id,time,bet,win
0,user_919,2019-01-01 14:06:51,NaN,NaN
1,user_973,2019-01-01 14:51:16,NaN,NaN
2,user_903,2019-01-01 16:31:16,NaN,NaN
3,user_954,2019-01-01 17:17:51,NaN,NaN
4,user_954,2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
995,user_984,2019-04-20 9:59:58,9754.0,NaN
996,,NaN,10054.0,29265.0
997,,NaN,10454.0,NaN
998,,NaN,1000.0,NaN
